<a href="https://colab.research.google.com/github/sc0les/nba_prediction_markets/blob/main/WIP_Neural_Net_Development_%26_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bayesian-optimization

In [ ]:
!pip install scikeras

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
full_df = pd.read_csv('/content/drive/My Drive/Sam Colelli BSTN Capstone/Datasets/final_nba_df.csv', index_col = 0)

In [ ]:
from sklearn.model_selection import train_test_split

WL_df = full_df.drop(columns = ['OU_hit', 'spread_hit'])

X = WL_df.iloc[:, :2818].to_numpy()
y = WL_df.iloc[:, 2818:].to_numpy().ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 1)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
score_acc = make_scorer(accuracy_score)

def nn_cl_bo(neurons, activation, optimizer, learning_rate,  batch_size, epochs ):
    optimizer_list = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    optimizer_dict = {'Adam': Adam(lr = learning_rate),
                 'SGD': SGD(lr = learning_rate),
                 'RMSprop': RMSprop(lr = learning_rate),
                 'Adadelta': Adadelta(lr = learning_rate),
                 'Adagrad': Adagrad(lr = learning_rate),
                 'Adamax': Adamax(lr = learning_rate),
                 'Nadam': Nadam(lr = learning_rate),
                 'Ftrl': Ftrl(lr=learning_rate)
                 }
    activation_list = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', LeakyReLU, 'relu']
    neurons = round(neurons)
    activation = activation_list[round(activation)]
    batch_size = round(batch_size)
    epochs = round(epochs)
    def nn_cl_fun():
        opt = Adam(lr = learning_rate)
        nn = Sequential()
        nn.add(Dense(neurons, input_dim = 2818, activation = activation))
        nn.add(Dense(neurons, activation = activation))
        #although it is a binary classification problem, I saw a big performance boost when changing it to sparse categorical with softmax, as opposed to sigmoid
        nn.add(Dense(2, activation='softmax'))
        nn.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
        return nn
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
    nn = KerasClassifier(model=nn_cl_fun, epochs=epochs, batch_size=batch_size, verbose=0)
    kfold = StratifiedKFold(n_splits=10, shuffle = True, random_state=1)
    score = cross_val_score(nn, X_train, y_train, scoring = score_acc, cv=kfold, fit_params={'callbacks':[es]}, error_score = 'raise').mean()
    return score

In [ ]:
# setting up parameters for GridSearch
params_nn ={
    'neurons': (10, 1000),
    'activation': (0, 9),
    'optimizer': (0,7),
    'learning_rate': (0.01, 1),
    'batch_size': (32, 1000),
    'epochs': (20, 100)
}
# Attempting to combine GridSearch hyperparameter selection and Bayesian Optimization so that I can save on computational time and cost
nn_bo = BayesianOptimization(nn_cl_bo, params_nn, random_state=1)
nn_bo.maximize(init_points=25, n_iter=4)

|   iter    |  target   | activa... | batch_... |  epochs   | learni... |  neurons  | optimizer |
-------------------------------------------------------------------------------------------------


| 1         | 0.4999    | 3.753     | 729.3     | 20.01     | 0.3093    | 155.3     | 0.6464    |
| 2         | 0.568     | 1.676     | 366.5     | 51.74     | 0.5434    | 425.0     | 4.797     |
| 3         | 0.5849    | 1.84      | 882.0     | 22.19     | 0.6738    | 423.1     | 3.911     |
| 4         | 0.4662    | 1.263     | 223.8     | 84.06     | 0.9686    | 320.3     | 4.846     |
| 5         | 0.5509    | 7.888     | 898.0     | 26.8      | 0.04866   | 178.1     | 6.147     |
| 6         | 0.5001    | 0.8851    | 439.6     | 96.63     | 0.5378    | 695.0     | 2.209     |
| 7         | 0.5171    | 6.179     | 839.9     | 21.46     | 0.7526    | 989.0     | 5.237     |
| 8         | 0.534     | 2.524     | 796.0     | 28.26     | 0.4534    | 909.5     | 2.055     |
| 9         | 0.5511    | 2.59      | 157.9     | 21.55     | 0.682     | 219.5     | 1.859     |
| 10        | 0.5002    | 4.424     | 83.65     | 65.93     | 0.1553    | 593.4     | 4.898     |
| 11        | 0.5002

In [ ]:
#displaying hyperparameters
params_nn_ = nn_bo.max['params']
activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', LeakyReLU, 'relu']
params_nn_['activation'] = activationL[round(params_nn_['activation'])]
params_nn_

{'activation': 'softplus',
 'batch_size': 882.0176784264352,
 'epochs': 22.191007455834093,
 'learning_rate': 0.6737628350766182,
 'neurons': 423.1317543434557,
 'optimizer': 3.9108287991202615}

In [ ]:
# Creating a second function to take the high-level hyperparameters to determine additional optimal parameters, such as number of hidden layers and dropoutrate
def nn_cl_bo2(neurons, activation, optimizer, learning_rate, batch_size, epochs, layers1, layers2, normalization, dropout, dropout_rate):
    optimizer_list = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    optimizer_dict = {'Adam': Adam(lr = learning_rate),
                 'SGD': SGD(lr = learning_rate),
                 'RMSprop': RMSprop(lr = learning_rate),
                 'Adadelta': Adadelta(lr = learning_rate),
                 'Adagrad': Adagrad(lr = learning_rate),
                 'Adamax': Adamax(lr = learning_rate),
                 'Nadam': Nadam(lr = learning_rate),
                 'Ftrl': Ftrl(lr = learning_rate)}
    activation_list = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', LeakyReLU,'relu']
    neurons = round(neurons)
    activation = activation_list[round(activation)]
    optimizer = optimizer_dict[optimizer_list[round(optimizer)]]
    batch_size = round(batch_size)
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    def nn_cl_fun():
        nn = Sequential()
        nn.add(Dense(neurons, input_dim = 2818, activation=activation))
        if normalization > 0.5:
            nn.add(BatchNormalization())
        for i in range(layers1):
            nn.add(Dense(neurons, activation=activation))
        if dropout > 0.5:
            nn.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
            nn.add(Dense(neurons, activation=activation))
        nn.add(Dense(2, activation = 'softmax'))
        nn.compile(loss = 'sparse_categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
        return nn
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
    nn = KerasClassifier(model=nn_cl_fun, epochs=epochs, batch_size=batch_size, verbose=0)
    kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 11)
    score = cross_val_score(nn, X_train, y_train, scoring = score_acc, cv = kfold, fit_params = {'callbacks':[es]}, error_score = 'raise').mean()
    return score

In [ ]:
params_nn2 ={
    'neurons': (10, 1000),
    'activation': (0, 9),
    'optimizer': (0,7),
    'learning_rate': (0.01, 1),
    'batch_size': (32, 1000),
    'epochs': (20, 100),
    'layers1': (1, 4),
    'layers2': (1, 4),
    'normalization': (0,1),
    'dropout': (0,1),
    'dropout_rate': (0,0.3)
}

nn_bo = BayesianOptimization(nn_cl_bo2, params_nn2, random_state=111)
nn_bo.maximize(init_points=25, n_iter=4)

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------
| 1         | 0.517     | 5.51      | 195.7     | 0.4361    | 0.2308    | 43.63     | 1.597     | 1.09      | 0.426     | 473.0     | 0.3377    | 6.935     |
| 2         | 0.5849    | 2.14      | 110.6     | 0.6696    | 0.1864    | 41.94     | 2.865     | 1.473     | 0.08322   | 1.758e+03 | 0.794     | 5.884     |
| 3         | 0.4151    | 7.337     | 991.2     | 0.5773    | 0.2441    | 53.71     | 1.11      | 2.817     | 0.1143    | 1.595e+03 | 0.6977    | 3.957     |
| 4         | 0.4491    | 2.468     | 998.5     | 0.138     | 0.1846    | 58.8      | 2.62      | 3.911     | 0.3296    | 787.1     | 0.319     | 6.631     |
| 5         | 0.4151    | 8.268     | 819.8     | 0.

In [ ]:
#mapping the second optimized GridSearch to the parameters that'll be used for model build and fit
params_nn_ = nn_bo.max['params']
learning_rate = params_nn_['learning_rate']
activation_list = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', LeakyReLU, 'relu']
params_nn_['activation'] = activation_list[round(params_nn_['activation'])]
params_nn_['batch_size'] = round(params_nn_['batch_size'])
params_nn_['epochs'] = round(params_nn_['epochs'])
params_nn_['layers1'] = round(params_nn_['layers1'])
params_nn_['layers2'] = round(params_nn_['layers2'])
params_nn_['neurons'] = round(params_nn_['neurons'])
optimizer_list = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','Adam']
optimizer_dict = {'Adam': Adam(lr = learning_rate),
                  'SGD': SGD(lr = learning_rate),
                  'RMSprop': RMSprop(lr = learning_rate),
                  'Adadelta': Adadelta(lr = learning_rate),
                  'Adagrad': Adagrad(lr = learning_rate),
                  'Adamax': Adamax(lr = learning_rate),
                  'Nadam': Nadam(lr = learning_rate),
                  'Ftrl': Ftrl(lr = learning_rate)
                  }
params_nn_['optimizer'] = optimizer_dict[optimizer_list[round(params_nn_['optimizer'])]]
params_nn_

{'activation': 'softplus',
 'batch_size': 111,
 'dropout': 0.6696002382466298,
 'dropout_rate': 0.1863728758202091,
 'epochs': 42,
 'layers1': 2,
 'layers2': 1,
 'learning_rate': 0.08321798850177216,
 'neurons': 902,
 'normalization': 0.7939625604796284,
 'optimizer': <keras.optimizers.optimizer_v2.nadam.Nadam at 0x7fe700160790>}

In [ ]:
#function for fitting the model to the optimized hyperparameters
def nn_cl_fun():
  nn = Sequential()
  nn.add(Dense(params_nn_['neurons'], input_dim = 2818, activation = params_nn_['activation']))
  nn.add(Dense(params_nn_['neurons'], input_dim = 2818, activation = params_nn_['activation']))
  if params_nn_['normalization'] > 0.5:
    nn.add(BatchNormalization())
  for i in range(params_nn_['layers1']):
    nn.add(Dense(params_nn_['neurons'], activation = params_nn_['activation']))
  if params_nn_['dropout'] > 0.5:
    nn.add(Dropout(params_nn_['dropout_rate'], seed=123))
  for i in range(params_nn_['layers2']):
    nn.add(Dense(params_nn_['neurons'], activation = params_nn_['activation']))
  nn.add(Dense(2, activation='softmax'))
  nn.compile(loss='sparse_categorical_crossentropy', optimizer = params_nn_['optimizer'], metrics = ['accuracy'])
  return nn

es = EarlyStopping(monitor = 'accuracy', mode = 'max', verbose = 0, patience = 20)
nn = KerasClassifier(build_fn = nn_cl_fun, epochs = params_nn_['epochs'], batch_size = params_nn_['batch_size'], verbose = 0)
nn.fit(X_train_scaled, y_train, validation_data = (X_test_scaled, y_test), verbose = 1)

Epoch 1/42
138/138 [==============================] - 3s 10ms/step - loss: 2598938.0000 - accuracy: 0.5720 - val_loss: 0.6806 - val_accuracy: 0.5912
Epoch 2/42
138/138 [==============================] - 1s 8ms/step - loss: 0.7036 - accuracy: 0.5790 - val_loss: 0.9706 - val_accuracy: 0.5912
Epoch 3/42
138/138 [==============================] - 1s 9ms/step - loss: 1.5490 - accuracy: 0.5793 - val_loss: 0.6770 - val_accuracy: 0.5912
Epoch 4/42
138/138 [==============================] - 1s 8ms/step - loss: 0.6929 - accuracy: 0.5798 - val_loss: 0.6832 - val_accuracy: 0.5836
Epoch 5/42
138/138 [==============================] - 1s 8ms/step - loss: 1.4213 - accuracy: 0.5794 - val_loss: 0.6774 - val_accuracy: 0.5912
Epoch 6/42
138/138 [==============================] - 1s 8ms/step - loss: 0.8359 - accuracy: 0.5798 - val_loss: 0.6874 - val_accuracy: 0.5870
Epoch 7/42
138/138 [==============================] - 1s 9ms/step - loss: 0.6887 - accuracy: 0.5810 - val_loss: 0.6826 - val_accuracy: 0.5912

KerasClassifier(
	model=None
	build_fn=<function nn_cl_fun at 0x7fe6ff7a4d40>
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=111
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=42
	class_weight=None
)

In [ ]:
#the remainder of this notebook is trial and error based on the above hyperparameters, combined with general, out-of-the-box layering and optimization functions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [ ]:
model = Sequential()
model.add(Dense(1950, activation= 'elu'))
model.add(Dense(1048, activation= 'elu'))
model.add(Dropout(0.13))
model.add(Dense(512, activation= 'elu'))
model.add(Dense(256, activation= 'elu'))
model.add(Dense(128, activation= 'elu'))
model.add(Dense(2, activation= 'softmax'))

In [ ]:
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

model.fit(X_train_scaled, y_train, epochs = 55, validation_split = 0.3)
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)

Epoch 1/55
335/335 [==============================] - 3s 6ms/step - loss: 0.7299 - accuracy: 0.6226 - val_loss: 0.6601 - val_accuracy: 0.6199
Epoch 2/55
335/335 [==============================] - 2s 5ms/step - loss: 0.6176 - accuracy: 0.6605 - val_loss: 0.6469 - val_accuracy: 0.6395
Epoch 3/55
335/335 [==============================] - 2s 5ms/step - loss: 0.5930 - accuracy: 0.6846 - val_loss: 0.6195 - val_accuracy: 0.6674
Epoch 4/55
335/335 [==============================] - 2s 5ms/step - loss: 0.5679 - accuracy: 0.7105 - val_loss: 0.6343 - val_accuracy: 0.6761
Epoch 5/55
335/335 [==============================] - 2s 5ms/step - loss: 0.5479 - accuracy: 0.7208 - val_loss: 0.6488 - val_accuracy: 0.6709
Epoch 6/55
335/335 [==============================] - 2s 5ms/step - loss: 0.4918 - accuracy: 0.7570 - val_loss: 0.6469 - val_accuracy: 0.6953
Epoch 7/55
335/335 [==============================] - 2s 5ms/step - loss: 0.4539 - accuracy: 0.7796 - val_loss: 0.6962 - val_accuracy: 0.6593
Epoch 

In [ ]:
model = Sequential()

model.add(Dense(1048, activation= 'relu'))
model.add(Dense(512, activation= 'relu'))
model.add(Dense(256, activation= 'relu'))
model.add(Dense(128, activation= 'relu'))
model.add(Dense(2, activation= 'sigmoid'))

In [ ]:
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

model.fit(X_train_scaled, y_train, epochs = 59, validation_split = 0.3)
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)

Epoch 1/59
335/335 [==============================] - 3s 6ms/step - loss: 0.6760 - accuracy: 0.6303 - val_loss: 0.6164 - val_accuracy: 0.6589
Epoch 2/59
335/335 [==============================] - 2s 5ms/step - loss: 0.6149 - accuracy: 0.6677 - val_loss: 0.6274 - val_accuracy: 0.6380
Epoch 3/59
335/335 [==============================] - 2s 5ms/step - loss: 0.5845 - accuracy: 0.6888 - val_loss: 0.6264 - val_accuracy: 0.6724
Epoch 4/59
335/335 [==============================] - 2s 5ms/step - loss: 0.5438 - accuracy: 0.7263 - val_loss: 0.6483 - val_accuracy: 0.6646
Epoch 5/59
335/335 [==============================] - 2s 5ms/step - loss: 0.4949 - accuracy: 0.7552 - val_loss: 0.6772 - val_accuracy: 0.6717
Epoch 6/59
335/335 [==============================] - 2s 5ms/step - loss: 0.4291 - accuracy: 0.8010 - val_loss: 0.6855 - val_accuracy: 0.6604
Epoch 7/59
335/335 [==============================] - 2s 5ms/step - loss: 0.3552 - accuracy: 0.8383 - val_loss: 0.7556 - val_accuracy: 0.6746
Epoch 

In [ ]:
#relu, softplus
#ftrl, adam

combined_params = {
    'activation': 'relu',
    'batch_size': 32,
    'dropout': 0.6696002382466298,
    'dropout_rate': 0.1863728758202091,
    'epochs': 59,
    'layers1': 3,
    'layers2': 4,
    'learning_rate': 0.32958578622954593,
    'neurons': 787,
    'normalization': 0.3189664036607044,
    'optimizer': 'adam'
    }

def nn_cl_fun():
  nn = Sequential()
  nn.add(Dense(1950, input_dim=2915, activation=combined_params['activation']))
  nn.add(Dense(combined_params['neurons'], activation=combined_params['activation']))
  if combined_params['normalization'] > 0.5:
    nn.add(BatchNormalization())
  for i in range(combined_params['layers1']):
    nn.add(Dense(combined_params['neurons'], activation=combined_params['activation']))
    nn.add(Dropout(combined_params['dropout_rate']))
  for i in range(combined_params['layers2']):
    nn.add(Dense(combined_params['neurons'], activation=combined_params['activation']))
  nn.add(Dense(2, activation='softmax'))
  nn.compile(loss='sparse_categorical_crossentropy', optimizer = combined_params['optimizer'], metrics=['accuracy'])
  return nn

es = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
#nn = KerasClassifier(model=nn_cl_fun, epochs=combined_params['epochs'], batch_size=combined_params['batch_size'],verbose=0)
nn.fit(X_train_scaled, y_train, epochs=combined_params['epochs'],  validation_data = (X_test_scaled, y_test), verbose=1)
#test_loss, test_acc = nn.evaluate(X_test_scaled, y_test)

Epoch 1/59
138/138 [==============================] - 3s 10ms/step - loss: 2456178.5000 - accuracy: 0.5453 - val_loss: 0.7722 - val_accuracy: 0.5802
Epoch 2/59
138/138 [==============================] - 1s 9ms/step - loss: 0.7613 - accuracy: 0.5527 - val_loss: 1.2731 - val_accuracy: 0.5912
Epoch 3/59
138/138 [==============================] - 1s 9ms/step - loss: 0.7244 - accuracy: 0.5536 - val_loss: 4.8944 - val_accuracy: 0.5912
Epoch 4/59
138/138 [==============================] - 1s 8ms/step - loss: 0.7142 - accuracy: 0.5587 - val_loss: 60.1787 - val_accuracy: 0.5110
Epoch 5/59
138/138 [==============================] - 1s 9ms/step - loss: 1.0798 - accuracy: 0.5591 - val_loss: 11.2603 - val_accuracy: 0.5073
Epoch 6/59
138/138 [==============================] - 1s 8ms/step - loss: 0.7012 - accuracy: 0.5613 - val_loss: 52.3655 - val_accuracy: 0.5154
Epoch 7/59
138/138 [==============================] - 1s 9ms/step - loss: 0.7048 - accuracy: 0.5656 - val_loss: 1216.6718 - val_accuracy: 

KerasClassifier(
	model=None
	build_fn=<function nn_cl_fun at 0x7fe6ff7a4d40>
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=111
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=42
	class_weight=None
)